In [5]:
import urllib.request
from bs4 import BeautifulSoup
from flask import Flask, render_template, request
import sqlite3

app = Flask(__name__)

conn = sqlite3.connect("music.db")
cursor = conn.cursor()
cursor.execute("DELETE FROM test")

conn.commit()

In [6]:
na = "강남"

In [7]:
# url = "https://www.youtube.com/results?search_query={}".format(na)

# print(url)




def chart_balad_save():
    url = "https://www.melon.com/chart/day/index.htm?classCd=GN0500" # 멜론 발라드 차트 url 
    hdr = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.5005.61 Safari/537.36"}
    req = urllib.request.Request(url, headers=hdr)
    html = urllib.request.urlopen(req).read()
    soup = BeautifulSoup(html, 'html.parser')

    conn = sqlite3.connect("music.db")
    cursor = conn.cursor()


    
    lst50 = soup.select('.lst50, .lst100')

    genre = []

    paging = soup.find('div', class_ = 'calendar_prid').get_text().replace("\n","")
    genre.append([paging])



    melonList = []
    for i in lst50:
        ranking = (i.select_one("td > div > span.rank").text+"위") # 멜론 사이트에서 발라드 순위 가져와서 temp에 추가
        title = (i.select_one("td > div > div > div.ellipsis.rank01").a.text) # 멜론 사이트에서 노래 제목 가져오서 temp에 추가
        singer = (i.select_one('td > div > div > div.ellipsis.rank02').a.text) # 멜론 사이트에서 가수 이름 가져와서 temp에 추가
        album = (i.select_one('td > div > div > div.ellipsis.rank03').a.text) # 멜론 사이트에서 앨범 이름 가져와서 temp에 주가
        url_title = "https://www.youtube.com/results?search_query={}".format((i.select_one('td > div > div > div.ellipsis.rank03').a.text))
        url_singer = "https://www.youtube.com/results?search_query={}".format((i.select_one('td > div > div > div.ellipsis.rank02').a.text))
        melonList.append([ranking, title, singer, album]) # 멜론리스트에 temp에 있는 것들 추가
        
        inser='insert into test (ranking, title, singer, album, youtube_title, youtube_singer) values(?,?,?,?,?,?)'
        cursor.execute(inser,(ranking, title, singer, album, url_title, url_singer))
    conn.commit()


chart_balad_save()


In [ ]:
conn.close()